# Treinamento e Avaliação de Modelos para Previsão de Risco de Enchente

Este notebook tem como objetivo treinar e avaliar dois modelos de classificação (Logistic Regression e Random Forest) para prever o `nivel_risco` de enchente com base no `acumulado_chuva_1_h_mm` e `cod_estacao` dos dados processados do CEMADEN.

## 1. Configuração Inicial e Carregamento de Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Configurações de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Carregar os dados processados
df = pd.read_csv('../data/cemaden_official_processed_hourly.csv')
df.head()

In [ ]:
df.info()

## 2. Análise Exploratória Breve

In [ ]:
# Distribuição do target (nivel_risco)
print(df['nivel_risco'].value_counts(normalize=True) * 100)
sns.countplot(x='nivel_risco', data=df)
plt.title('Distribuição da Variável Alvo (nivel_risco)')
plt.show()

In [ ]:
# Relação entre acumulado_chuva_1_h_mm e nivel_risco
sns.boxplot(x='nivel_risco', y='acumulado_chuva_1_h_mm', data=df)
plt.title('Acumulado de Chuva (1h) por Nível de Risco')
plt.show()

## 3. Preparação dos Dados para Modelagem

In [ ]:
# Seleção de Features (X) e Target (y)
# Vamos incluir 'cod_estacao' como feature categórica
X = df[['acumulado_chuva_1_h_mm', 'cod_estacao']]
y = df['nivel_risco']

# Definir o transformador para One-Hot Encoding da coluna 'cod_estacao'
# remainder='passthrough' mantém as outras colunas (acumulado_chuva_1_h_mm)
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['cod_estacao'])
    ],
    remainder='passthrough'
)

# Divisão em conjuntos de treino e teste (80/20 estratificado)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Formato de X_train: {X_train.shape}")
print(f"Formato de X_test: {X_test.shape}")
print(f"Formato de y_train: {y_train.shape}")
print(f"Formato de y_test: {y_test.shape}")

print("\nDistribuição do nivel_risco no conjunto de treino:")
print(y_train.value_counts(normalize=True) * 100)

print("\nDistribuição do nivel_risco no conjunto de teste:")
print(y_test.value_counts(normalize=True) * 100)

## 4. Modelo 1: Logistic Regression

In [ ]:
# Criar o pipeline para Logistic Regression com pré-processamento
pipeline_log_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear', max_iter=1000))
])

# Treinar o modelo
pipeline_log_reg.fit(X_train, y_train)

# Predições no conjunto de teste
y_pred_log_reg_test = pipeline_log_reg.predict(X_test)

# Predições no conjunto de treino (para verificar overfitting)
y_pred_log_reg_train = pipeline_log_reg.predict(X_train)

# Avaliação no conjunto de Teste
print("Logistic Regression - Avaliação no CONJUNTO DE TESTE")
print("Acurácia:", accuracy_score(y_test, y_pred_log_reg_test))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_log_reg_test))
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_log_reg_test))

# Avaliação no conjunto de Treino
print("\nLogistic Regression - Avaliação no CONJUNTO DE TREINO")
print("Acurácia:", accuracy_score(y_train, y_pred_log_reg_train))
print("Relatório de Classificação:")
print(classification_report(y_train, y_pred_log_reg_train))

## 5. Modelo 2: Random Forest

In [ ]:
# Criar o pipeline para Random Forest com pré-processamento
pipeline_rf_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=100))
])

# Treinar o modelo
pipeline_rf_clf.fit(X_train, y_train)

# Predições no conjunto de teste
y_pred_rf_clf_test = pipeline_rf_clf.predict(X_test)

# Predições no conjunto de treino (para verificar overfitting)
y_pred_rf_clf_train = pipeline_rf_clf.predict(X_train)

# Avaliação no conjunto de Teste
print("Random Forest - Avaliação no CONJUNTO DE TESTE")
print("Acurácia:", accuracy_score(y_test, y_pred_rf_clf_test))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_rf_clf_test))
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_rf_clf_test))

# Avaliação no conjunto de Treino
print("\nRandom Forest - Avaliação no CONJUNTO DE TREINO")
print("Acurácia:", accuracy_score(y_train, y_pred_rf_clf_train))
print("Relatório de Classificação:")
print(classification_report(y_train, y_pred_rf_clf_train))

# Importância das Features (apenas para Random Forest e após o fit do pipeline)
try:
    # Obter o transformador OneHotEncoder do pipeline
    onehot_transformer = pipeline_rf_clf.named_steps['preprocessor'].named_transformers_['onehot']
    onehot_features = onehot_transformer.get_feature_names_out(['cod_estacao'])
    
    # As features numéricas são passadas através do 'remainder'
    numeric_features = [col for col in X_train.columns if col not in ['cod_estacao']]
    
    # Combinar nomes das features na ordem correta
    all_feature_names = np.concatenate([onehot_features, numeric_features])
    
    importances = pipeline_rf_clf.named_steps['classifier'].feature_importances_
    
    if len(all_feature_names) == len(importances):
        feature_importance_df = pd.DataFrame({'feature': all_feature_names, 'importance': importances})
        feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
        
        print("\nImportância das Features (Random Forest):")
        print(feature_importance_df.head(10)) # Mostrar top 10 features
        
        plt.figure(figsize=(12, max(6, len(feature_importance_df.head(10)) * 0.5)))
        sns.barplot(x='importance', y='feature', data=feature_importance_df.head(10), palette='viridis')
        plt.title('Top 10 Features Mais Importantes - Random Forest')
        plt.tight_layout()
        plt.show()
    else:
        print(f"Erro: Número de nomes de features ({len(all_feature_names)}) não corresponde ao número de importâncias ({len(importances)}).")
        print("Nomes das features extraídos:", all_feature_names)
except Exception as e:
    print(f"Erro ao calcular ou exibir importância das features: {e}")

## 6. Comparação dos Modelos e Conclusão

In [ ]:
# Coletar métricas para comparação (do conjunto de TESTE)
metrics_log_reg_test = classification_report(y_test, y_pred_log_reg_test, output_dict=True)
metrics_rf_clf_test = classification_report(y_test, y_pred_rf_clf_test, output_dict=True)

# Coletar métricas do conjunto de TREINO para verificar overfitting
metrics_log_reg_train = classification_report(y_train, y_pred_log_reg_train, output_dict=True)
metrics_rf_clf_train = classification_report(y_train, y_pred_rf_clf_train, output_dict=True)

comparison_data = {
    'Modelo': ['Logistic Regression', 'Random Forest'],
    'Acurácia (Teste)': [accuracy_score(y_test, y_pred_log_reg_test), accuracy_score(y_test, y_pred_rf_clf_test)],
    'Acurácia (Treino)': [accuracy_score(y_train, y_pred_log_reg_train), accuracy_score(y_train, y_pred_rf_clf_train)],
    'F1-Score (Teste, weighted)': [metrics_log_reg_test['weighted avg']['f1-score'], metrics_rf_clf_test['weighted avg']['f1-score']],
    'F1-Score (Treino, weighted)': [metrics_log_reg_train['weighted avg']['f1-score'], metrics_rf_clf_train['weighted avg']['f1-score']],
    'Recall (Teste, Nível 2)': [metrics_log_reg_test.get('2', {}).get('recall', 0.0), metrics_rf_clf_test.get('2', {}).get('recall', 0.0)],
    'Precisão (Teste, Nível 2)': [metrics_log_reg_test.get('2', {}).get('precision', 0.0), metrics_rf_clf_test.get('2', {}).get('precision', 0.0)]
}

df_comparison = pd.DataFrame(comparison_data)
print("Tabela Comparativa dos Modelos (Métricas de Teste e Treino):")
print(df_comparison.round(4))

### Conclusão Preliminar

Com base na tabela acima, podemos discutir:
1. Qual modelo apresentou melhor desempenho geral no conjunto de teste?
2. Qual se saiu melhor na identificação dos casos de risco mais elevado (Nível 2) no teste?
3. Há sinais de overfitting (grande diferença entre métricas de treino e teste)?
4. Qual o impacto da inclusão da feature `cod_estacao` (analisando a importância das features do Random Forest)?

Esta análise ajudará a decidir qual modelo (e pipeline de pré-processamento) será salvo e utilizado no script `2_train_model.py`.

### Salvando o Modelo Escolhido (Exemplo)

Supondo que o Random Forest (com o pipeline) seja o escolhido:

In [ ]:
# Exemplo: Salvando o pipeline completo do Random Forest
# model_to_save = pipeline_rf_clf 
# model_dir = '../ml_model'
# if not os.path.exists(model_dir):
#     os.makedirs(model_dir)
# model_filename = os.path.join(model_dir, 'cemaden_flood_risk_model_pipeline.joblib')
# joblib.dump(model_to_save, model_filename)
# print(f"Modelo (pipeline) salvo em {model_filename}")

# Descomente e ajuste conforme o modelo escolhido após a análise.